<a href="https://colab.research.google.com/github/callmeBze/Nepal_earthquake_map/blob/main/Nepal_earthquake_map.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np
import os
import requests
import folium
import branca.colormap as cm
import geopandas as gpd
from shapely.geometry import Point
import kagglehub



In [4]:
# 1. Dataset Download and Load

if 'path' not in locals():
    print("Downloading earthquake dataset...")
    path = kagglehub.dataset_download("amansinghnp/nepal-earthquake-seismicity-dataset-1990-2026")
    print(f"Dataset downloaded to: {path}")

data_file_name = 'nepal_earthquakes_1990_2026.csv'
data_file_path = os.path.join(path, data_file_name)

# Load the CSV file into a Pandas DataFrame
df = pd.read_csv(data_file_path)
print(f"Dataset '{data_file_name}' loaded successfully.")

Dataset 'nepal_earthquakes_1990_2026.csv' loaded successfully.


In [5]:
# 2. Data Preprocessing
# Convert 'time' column to datetime objects
df['time'] = pd.to_datetime(df['time'])

# Extract year, month, day, hour, and minute into new columns
df['year'] = df['time'].dt.year
df['month'] = df['time'].dt.month
df['day'] = df['time'].dt.day
df['hour'] = df['time'].dt.hour
df['minute'] = df['time'].dt.minute
print("Time-based features (year, month, day, hour, minute) extracted.")

Time-based features (year, month, day, hour, minute) extracted.


In [6]:
# 3. GeoJSON Download and Load
geojason_url = "https://raw.githubusercontent.com/johan/world.geo.json/master/countries/NPL.geo.json"
save_directory = "./data"
os.makedirs(save_directory, exist_ok=True)
geojason_file_name = "nepal_administrative_boundaries.geojson"
nepal_geojson_path = os.path.join(save_directory, geojason_file_name)

print(f"Attempting to download GeoJSON from: {geojason_url}")
try:
    response = requests.get(geojason_url)
    response.raise_for_status()
    with open(nepal_geojson_path, 'wb') as f:
        f.write(response.content)
    print(f"Downloaded GeoJSON to: {nepal_geojson_path}")
except requests.exceptions.RequestException as e:
    print(f"Error downloading GeoJSON file: {e}")
    raise # Re-raise to stop execution if GeoJSON is critical

# Load the downloaded GeoJSON file into a GeoDataFrame
geopandas_df = gpd.read_file(nepal_geojson_path)
print("GeoJSON file for Nepal loaded successfully.")

Attempting to download GeoJSON from: https://raw.githubusercontent.com/johan/world.geo.json/master/countries/NPL.geo.json
Downloaded GeoJSON to: ./data/nepal_administrative_boundaries.geojson
GeoJSON file for Nepal loaded successfully.


In [7]:
# 4. Convert Earthquake DataFrame to GeoDataFrame
geometry = [Point(xy) for xy in zip(df['longitude'], df['latitude'])]
earthquakes_gdf = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:4326")
print("Earthquakes DataFrame converted to GeoDataFrame.")

Earthquakes DataFrame converted to GeoDataFrame.


In [8]:
# 5. Perform Spatial Join
earthquakes_in_nepal = gpd.sjoin(earthquakes_gdf, geopandas_df, how="inner", predicate='within')
print(f"Number of earthquake events precisely within Nepal's administrative boundaries: {len(earthquakes_in_nepal)}")

Number of earthquake events precisely within Nepal's administrative boundaries: 685


In [9]:
# 6. Folium Map Generation with Discretized Magnitude Legend and Attribution

# Approximate geographical coordinates of Nepal for map centering
nepal_latitude = 28.3949
nepal_longitude = 84.1240

# Create a folium.Map object centered on Nepal
nepal_map = folium.Map(location=[nepal_latitude, nepal_longitude], zoom_start=7)

In [10]:
7. # Define magnitude classification function
def classify_magnitude(mag):
    if mag < 4.0:
        return "< 4.0 (Minor)"
    elif 4.0 <= mag < 4.5:
        return "4.0 - 4.4 (Light)"
    elif 4.5 <= mag < 5.0:
        return "4.5 - 4.9 (Light-Moderate)"
    elif 5.0 <= mag < 5.5:
        return "5.0 - 5.4 (Moderate)"
    elif 5.5 <= mag < 6.0:
        return "5.5 - 5.9 (Strong)"
    else: # mag >= 6.0
        return "≥ 6.0 (Major-Great)"

# Apply classification to create a new 'mag_class' column
earthquakes_in_nepal['mag_class'] = earthquakes_in_nepal['mag'].apply(classify_magnitude)

# Define colors for each magnitude class
mag_colors = {
    "< 4.0 (Minor)": "#a6cee3",     # Light Blue
    "4.0 - 4.4 (Light)": "#1f78b4", # Blue
    "4.5 - 4.9 (Light-Moderate)": "#b2df8a", # Light Green
    "5.0 - 5.4 (Moderate)": "#33a02c", # Green
    "5.5 - 5.9 (Strong)": "#fb9a99",  # Light Red
    "≥ 6.0 (Major-Great)": "#e31a1c"   # Red
}

# Ensure all classification labels are in order for the legend
mag_class_order = [
    "< 4.0 (Minor)",
    "4.0 - 4.4 (Light)",
    "4.5 - 4.9 (Light-Moderate)",
    "5.0 - 5.4 (Moderate)",
    "5.5 - 5.9 (Strong)",
    "≥ 6.0 (Major-Great)"
]


In [11]:
8. # Add CircleMarkers for each earthquake event within Nepal's territory
for index, row in earthquakes_in_nepal.iterrows():
    # Determine the color based on the magnitude class
    color_for_event = mag_colors.get(row['mag_class'], '#808080') # Default to grey if class not mapped

    # Create a popup message with relevant earthquake information
    popup_html = f"""
        <b>Magnitude Class:</b> {row['mag_class']}<br>
        <b>Magnitude:</b> {row['mag']}<br>
        <b>Depth:</b> {row['depth']} km<br>
        <b>Place:</b> {row['place']}<br>
        <b>Time:</b> {row['time'].strftime('%Y-%m-%d %H:%M:%S')}
    """

    # Add a CircleMarker for each earthquake event
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=row['mag'] * .5, # Adjust radius based on magnitude for better visualization
        color=color_for_event,
        fill=True,
        fill_color=color_for_event,
        fill_opacity=0.7,
        tooltip=f"Mag: {row['mag']}, Class: {row['mag_class']}", # Tooltip for quick info on hover
        popup=folium.Popup(popup_html, max_width=300)
    ).add_to(nepal_map)

In [12]:
9. #  Custom HTML Legend for Discretized Magnitude Classes
legend_html = '''
     <div style="position: fixed;
                 bottom: 50px; left: 50px; width: 200px; height: auto;
                 border:2px solid grey; z-index:9999; font-size:14px;
                 background-color:white; opacity:0.9;">
       &nbsp; <b>Magnitude Class</b> <br>
       '''

for label in mag_class_order:
    color = mag_colors[label]
    legend_html += f'''
        &nbsp; <i style="background:{color}; border:1px solid gray;">&nbsp;&nbsp;&nbsp;&nbsp;</i> &nbsp; {label} <br>
    '''
legend_html += '''
    </div>
    '''
nepal_map.get_root().html.add_child(folium.Element(legend_html))

# --- Add Creator Attribution ---
user_name = "Bijaya Puri"
attribution_html = f'''
    <div style="position: fixed;
                bottom: 10px; right: 10px; width: 150px; height: auto;
                border:1px solid lightgrey; z-index:9999; font-size:10px;
                background-color:white; opacity:0.8;">
        &nbsp; Created by {user_name} &nbsp;
    </div>
    '''
nepal_map.get_root().html.add_child(folium.Element(attribution_html))

# Display the map
display(nepal_map)

In [14]:
nepal_map.save('earthquakes_in_nepal_by_Bijaya_Puri.html')